# Get Data

In [2]:
import pandas as pd
import ast
import keyboard

# dataLamaPath = 'data/new-fix-data.csv'
dataBaruPath = 'data/tempat-wisata-di-yogyakarta-overview.csv'
dataByPlace = 'data/byPlaceData.csv'

# dfOld = pd.read_csv(dataLamaPath)
dfNew = pd.read_csv(dataBaruPath)

dfByPlace = pd.read_csv(dataByPlace)

In [ ]:
dfNew.head()

In [ ]:
dfByPlace.head()

# Data Assessing

In [ ]:
# Data Baru
dfNew.columns

In [ ]:
dfByPlace.columns

Data yang diambil: 
- name
- address
- coordinates
- workday_timing
- closed_on
- reviews
- rating
- most_popular_times
- popular_times
- reviewer_name
- rating_review
- review_text
- published_at_date
- accessibility_enabled
- planning_enabled
- children_enabled

## Cek Informasi Kolom

## Merampingkan Data

In [ ]:
# grpByReviewer = dfOld.groupby('name').agg({
#     'address': lambda x: list(x),
#     'coordinates': lambda x: list(x),
#     'workday_timing': lambda x: list(x),
#     'closed_on': lambda x: list(x),
#     'reviews': lambda x: list(x),
#     'rating': lambda x: list(x),
#     'most_popular_times': lambda x: list(x),
#     'popular_times': lambda x: list(x),
#     'popular_times': lambda x: list(x),
#     'reviewer_name': lambda x: list(x),
#     'rating_review': lambda x: list(x),
#     'review_text': lambda x: list(x),
#     'published_at_date': lambda x: list(x),
#     'accessibility_enabled': lambda x: list(x),
#     'planning_enabled': lambda x: list(x),
#     'children_enabled': lambda x: list(x)
# }).reset_index()

# grpByReviewer.to_csv('data/byPlaceData.csv', index=False)

## Normalisasi Bentuk

Percobaan

In [ ]:
def process_lists_in_dataframe(df, columns_to_normalize):
    # Loop melalui setiap kolom yang perlu dinormalisasi dalam DataFrame
    for column in columns_to_normalize:
        if column in df.columns:
            # Loop melalui setiap baris dalam kolom yang relevan
            for i in range(len(df[column])):
                if isinstance(df[column].iloc[i], str):
                    try:
                        # Convert string representation of list to actual list
                        df[column].iloc[i] = ast.literal_eval(df[column].iloc[i])
                    except (ValueError, SyntaxError):
                        continue
                
                if isinstance(df[column].iloc[i], list):
                    # Convert list to a set to check for duplicates
                    value_counts = pd.Series(df[column].iloc[i]).value_counts()
                    
                    # Check if there are any duplicates (i.e., any value count >= 2)
                    if any(value_counts >= 1):
                        # Replace the list with the first element
                        df.at[i, column] = df[column].iloc[i][0]
    
    return df

columns_to_normalize = ['address', 'coordinates', 'workday_timing', 'closed_on','rating',
                        'reviews', 'most_popular_times', 'popular_times', 
                        'accessibility_enabled', 'planning_enabled', 'children_enabled']

dfByPlace = process_lists_in_dataframe(dfByPlace, columns_to_normalize)

In [ ]:
dfByPlace.head()

In [ ]:
# # Percobaan 4

# chunksize = 10000  # Ukuran chunks (dapat disesuaikan)

# # Kolom yang berbentuk list dan perlu dipisahkan ke dalam baris yang terpisah
# list_columns = [
#     'coordinates', 'workday_timing', 'closed_on', 'reviews',
#     'most_popular_times', 'popular_times', 'accessibility_enabled',
#     'planning_enabled', 'children_enabled'
# ]

# # Membuat file output kosong
# output_file_path = 'data/normalized_byPlaceData.csv'
# header_written = False

# # Proses per chunk
# for chunk in pd.read_csv('data/byPlaceData.csv', chunksize=chunksize):
#     for column in list_columns:
#         # Memisahkan kolom yang berisi list
#         chunk[column] = chunk[column].apply(lambda x: x.split(',') if isinstance(x, str) else [])
#         chunk = chunk.explode(column).reset_index(drop=True)
    
#     # Menghapus duplikasi
#     chunk = chunk.drop_duplicates()

#     # Simpan chunk yang telah diproses ke file CSV
#     if not header_written:
#         chunk.to_csv(output_file_path, mode='w', index=False, header=True)
#         header_written = True
#     else:
#         chunk.to_csv(output_file_path, mode='a', index=False, header=False)

# print(f"Data yang telah dinormalisasi telah disimpan di {output_file_path}")

## Ambil Data 

### Rating 

In [ ]:
iterator = 0

def update_rating(data1, data2, iterator):
    # Loop melalui setiap baris di data1
    for idx, row in data2.iterrows():
        # Cari baris di data2 dengan nama_wisata yang sama
        matching_row = data1[data1['name'] == row['name']]
        
        if not matching_row.empty:
            print('Ada data', row['name'])
            # Jika ada kecocokan, perbarui jumlah_reviews di data1
            data1.at[idx, 'rating'] = matching_row['rating'].values[0]
            data1.at[idx, 'is_rating_updated'] = 1
            iterator = iterator + 1
        else:
            print('Tidak ada data', row['name'])
            data1.at[idx, 'is_rating_updated'] = 0
    
    # Kembalikan data1 yang sudah diperbarui
    return data1, iterator

updatedDf, iterator = update_rating(dfByPlace, dfNew, iterator)
print(iterator)
updatedDf.head()


### Jumlah Reviews 

In [ ]:
dfOld[dfOld['name'] == 'Taman Pintar Yogyakarta'].reviews.sample(1)

In [ ]:
iterator = 0

def update_reviews(data1, data2, iterator):
    # Loop melalui setiap baris di data1
    for idx, row in data2.iterrows():
        # Cari baris di data2 dengan nama_wisata yang sama
        matching_row = data1[data1['name'] == row['name']]
        
        if not matching_row.empty:
            print('Ada data', row['name'])
            # Jika ada kecocokan, perbarui jumlah_reviews di data1
            data1.at[idx, 'reviews'] = matching_row['reviews'].values[0]
            data1.at[idx, 'is_reviews_updated'] = 1
            iterator = iterator + 1
        else:
            print('Tidak ada data', row['name'])
            data1.at[idx, 'is_reviews_updated'] = 0
    
    # Kembalikan data1 yang sudah diperbarui
    return data1, iterator

updatedDf, iterator = update_reviews(dfByPlace, dfNew, iterator)
print(iterator)
updatedDf.head()


In [ ]:
dfByPlace.to_csv('data/byPlaceData.csv')

# Data Cleaning

## Duplicated Data

In [3]:
dfByPlace.duplicated().sum()

np.int64(0)

## Data Null

In [4]:
dfByPlace.isna().sum()

Unnamed: 0                   0
name                         0
address                      0
coordinates                  0
workday_timing               0
closed_on                    0
reviews                      0
rating                       0
most_popular_times           0
popular_times                0
reviewer_name                0
rating_review                0
review_text                  0
published_at_date            0
accessibility_enabled        0
planning_enabled             0
children_enabled             0
is_rating_updated        13881
is_reviews_updated       13881
dtype: int64

## Content Selection

In [5]:
# Tentukan range data yang ingin diiterasi (misal: setengah dari total data)
total_rows = len(dfByPlace)
half_range =  10# Mengambil setengah dari total datatotal_rows // 2 

# Ambil subset dari DataFrame berdasarkan range yang diinginkan
df_subset = dfByPlace.iloc[:half_range]  # Mengambil baris dari indeks 0 hingga half_range


# List untuk menyimpan indeks baris yang dipilih untuk dipertahankan
rows_to_keep = []

print("Mulai iterasi melalui setiap baris. Tekan Enter untuk mempertahankan, Backspace untuk menghapus.")

# Iterasi melalui setiap baris dalam DataFrame
for index, row in dfByPlace.iterrows():
    print(f"Nama: {row['name']}")

    # Menunggu input dari keyboard
    while True:
        if keyboard.read_event().event_type == 'down':  # Menunggu tombol ditekan
            if keyboard.is_pressed('enter'):
                print("Dipertahankan.")
                rows_to_keep.append(index)  # Menyimpan indeks baris yang dipertahankan
                break  # Keluar dari loop dan lanjutkan ke baris berikutnya

            elif keyboard.is_pressed('backspace'):
                
                
                print("Dihapus.")
                break  # Keluar dari loop dan lanjutkan ke baris berikutnya

# Memfilter DataFrame berdasarkan baris yang dipertahankan
dfFiltered = dfByPlace.loc[rows_to_keep]
dfFiltered.to_csv('data/coba.csv')

Mulai iterasi melalui setiap baris. Tekan Enter untuk mempertahankan, Backspace untuk menghapus.
Nama:  ( Alun Alun Utara)
Dipertahankan.
Nama:  Embung Kladuan UII
Dipertahankan.
Nama:  Jembatan bok bang
Dihapus.
Nama:  Sendang Sari Mulyo
Dipertahankan.
Nama:  Stupa Buddha Dawangsari
Dipertahankan.
Nama:  Sumber Mata Air Bengkung
Dipertahankan.
Nama:  Wisata Air Kali Progo
Dipertahankan.
Nama:  alkid
Dipertahankan.
Nama: " Wisata khayangan Watu Kendhil "
Dipertahankan.
Nama: "AOM Fishing" Tempat Pemancingan
Dipertahankan.
Nama: "Kampung Maggot" Green Gunter Farm
Dipertahankan.
Nama: "PRAMBANAN" Spot View
Dipertahankan.
Nama: "SB Enterprise" Pekalongan
Dipertahankan.
Nama: "SH" Spot Hunting Fly over Kramasan
Dipertahankan.
Nama: "Setya Jaya" Farm 
Dipertahankan.
Nama: "UMBUL SEWU PENGGING" Wahana asyik bermain air keluarga bahagia
Dipertahankan.
Nama: "WILiKAH" Wisata Lima Langkah Serading
Dipertahankan.
Nama: (Mangkujo) Magelang Kuda & Joglo
Dipertahankan.
Nama: * Kampoeng tudong *
Dip

## Feature Selection